In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import cv2
import os
import random


In [44]:
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
test_df=pd.read_csv('/content/drive/MyDrive/student_resource 3/dataset/test.csv')
data=pd.read_csv('/content/drive/MyDrive/student_resource 3/dataset/train.csv')

In [46]:
test_df.head()

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth


In [47]:
data.head()

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram


In [48]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import re

# Step 1: Extract numerical values from the 'entity_value' column
# Using regular expression to capture numbers (can be improved with unit conversions)
def extract_numeric_value(val):
    match = re.search(r'(\d+\.?\d*)', val)
    return float(match.group(1)) if match else np.nan

data['numeric_value'] = data['entity_value'].apply(extract_numeric_value)

# Drop rows with missing values in 'numeric_value'
data_clean = data.dropna(subset=['numeric_value'])

# Step 2: Encode the 'entity_name' column using Label Encoding
label_encoder = LabelEncoder()
data_clean['entity_name_encoded'] = label_encoder.fit_transform(data_clean['entity_name'])

# Step 3: Select features and target variable
X = data_clean[['group_id', 'entity_name_encoded']]  # For simplicity, we are not using 'image_link'
y = data_clean['numeric_value']

# Step 4: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Train a RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 6: Make predictions and evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

rmse, y_pred[:5]  # Display RMSE and the first 5 predictions for evaluation


(27760706749495.504,
 array([2.33088509e+02, 1.24946545e+02, 6.04086186e+01, 2.98697679e+04,
        1.15931332e+05]))

In [55]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import numpy as np
import re

# Load and prepare data
# csv_file = "path_to_your_file.csv"  # Update this to the correct file path
# data = pd.read_csv(csv_file)

# Step 1: Extract numerical values from 'entity_value'
def extract_numeric_value(val):
    match = re.search(r'(\d+\.?\d*)', val)
    return float(match.group(1)) if match else None

data['numeric_value'] = data['entity_value'].apply(extract_numeric_value)

# Drop rows with missing numeric values
data_clean = data.dropna(subset=['numeric_value'])

# Encode 'entity_name' as a feature
label_encoder = LabelEncoder()
data_clean['entity_name_encoded'] = label_encoder.fit_transform(data_clean['entity_name'])

# Select features (excluding 'image_link') and target variable
X = data_clean[['group_id', 'entity_name_encoded']]  # Add other relevant features if necessary
y = data_clean['numeric_value']

# Step 2: Normalize the features and target variable
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)  # Scale the features
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))  # Scale the target

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Convert pandas DataFrame to TensorFlow dataset
def df_to_dataset(X, y, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(buffer_size=len(X))
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 32
train_dataset = df_to_dataset(X_train, y_train, batch_size)
test_dataset = df_to_dataset(X_test, y_test, batch_size)

# Define the regression model using Functional API
input_layer = tf.keras.layers.Input(shape=(X_train.shape[1],))
dense1 = tf.keras.layers.Dense(128, activation='relu')(input_layer)
dense2 = tf.keras.layers.Dense(64, activation='relu')(dense1)
output = tf.keras.layers.Dense(1)(dense2)  # Output layer for regression

# Create the model
model = tf.keras.Model(inputs=input_layer, outputs=output)

# Compile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='mean_squared_error',
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

# Train the model
epochs = 10
history = model.fit(train_dataset, validation_data=test_dataset, epochs=epochs)

# Evaluate the model
loss, rmse = model.evaluate(test_dataset)
print(f"Test RMSE: {rmse}")


Epoch 1/10
6597/6597 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - loss: 2.2950e-05 - root_mean_squared_error: 0.0040 - val_loss: 1.8973e-05 - val_root_mean_squared_error: 0.0044
Epoch 2/10
6597/6597 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 3.0954e-07 - root_mean_squared_error: 4.4189e-04 - val_loss: 1.9018e-05 - val_root_mean_squared_error: 0.0044
Epoch 3/10
6597/6597 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 1.8377e-06 - root_mean_squared_error: 8.5979e-04 - val_loss: 1.8977e-05 - val_root_mean_squared_error: 0.0044
Epoch 4/10
6597/6597 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 5.0242e-08 - root_mean_squared_error: 1.6424e-04 - val_loss: 1.9881e-05 - val_root_mean_squared_error: 0.0045
Epoch 5/10
6597/6597 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 1.4443e-07 - root_mean_squared_error: 2.5776e-04 - val_loss: 1.9017e-05 - val_root_mean_squared_error: 0.0044
Epoch 6/10
6597/6597 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - loss: 9.0433e-06 - root_mean_squared_error: 0.0027 - val_loss: 1.8970e-05 - val_ro

In [56]:
from sklearn.metrics import r2_score

# Get the model's predictions for the test set
y_pred = model.predict(X_test)

# Inverse transform the scaled predictions and actual values
y_pred_rescaled = scaler_y.inverse_transform(y_pred)
y_test_rescaled = scaler_y.inverse_transform(y_test)

# Calculate R-squared
r2 = r2_score(y_test_rescaled, y_pred_rescaled)
print(f"R-squared (R²): {r2}")


1650/1650 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step
R-squared (R²): -2.045080656376541e-05
